### Imports

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

### Dataset Exploration

In [ ]:
def explore_bbox_dataset(coco_annotations_file_path):
    with open(coco_annotations_file_path, "r") as f:
        coco_annotations = json.load(f)

    # number of images
    images_info = coco_annotations['images']
    num_images = len(images_info)
    print(f"Dataset contains {num_images} images.")

    # Image sizes
    image_sizes = {(img['height'], img['width']) for img in images_info}
    print(f"Dataset contains images of the following sizes: {list(image_sizes)}")

    # extracting the boundign boxes
    bboxes = np.round(np.array([img['bbox'] for img in coco_annotations['annotations']])).astype(int)


    for img in coco_annotations['annotations']:
        if len(img['bbox']) != 4:
            print("WTF ", len(img['bbox']))

    # width distribution
    widths = bboxes[:, 2]
    print('\nWidth:')
    stats(widths, "Width")

    # height distribution
    heights = bboxes[:, 3]
    print('\nHeight:')
    stats(heights, "Height")

    # Area distribution
    areas = widths * heights
    print("\nArea:")
    stats(areas, "Area")

    # Width to Height Ratio
    aspect_ratios = widths / heights
    print("\nAspect Ratio: ")
    stats(aspect_ratios, "Aspect Ratio")

    # Squareness
    squareness = np.min(bboxes[:, 2:],axis=1)**2 / areas
    print("\nSquareness: ")
    stats(squareness, 'Squareness')

    # Heat map of bbox location
    bbox_to_heat_map(bboxes, 640, 640)

def stats(data, title):
    min = np.min(data)
    max = np.max(data)
    mean = np.mean(data)
    median = np.median(data)
    var = np.var(data)
    sd= np.std(data) 

    print(f'\t- min: {np.round(min, 3)}')
    print(f'\t- max: {np.round(max, 3)}')
    print(f'\t- mean: {np.round(mean, 3)}')
    print(f'\t- median: {np.round(median, 3)}')
    print(f'\t- variance: {np.round(var, 3)}')
    print(f'\t- SD: {np.round(sd, 3)}')

    plt.hist(data, color='b')
    plt.title(title)
    plt.show()

          
def bbox_to_heat_map(bbox_data, width, height):
    num_boxes = bbox_data.shape[0]
    heat_map = np.zeros((num_boxes, width, height))

    for i in range(num_boxes):
        x = bbox_data[i][0]
        y = bbox_data[i][1]
        w = bbox_data[i][2]
        h = bbox_data[i][3]
        heat_map[i, x:x+w, y:y+h] = 1

    heat_map = np.mean(heat_map, axis=0)
    plt.imshow(heat_map, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title('Bounding Box Location Average')
    plt.show()

    coverage_area = heat_map.copy()
    coverage_area[coverage_area>0] = 1

    plt.imshow(coverage_area, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title('Bounding Box Coverage')
    plt.show()
     
    return heat_map


## Exploring Training Set

In [ ]:
train_coco = explore_bbox_dataset('./../datasets/tumor-segmentation/train/_annotations.coco.json')

## Explorating Validation Set

In [ ]:
explore_bbox_dataset('./../datasets/tumor-segmentation/validation/_annotations.coco.json')

## Exploring Test Set

In [ ]:
explore_bbox_dataset('./../datasets/tumor-segmentation/test/_annotations.coco.json')

## Generating Anchor Boxes
We need to ensure that all of the bounding boxes can be detected by at least 1 anchor with a sufficient IoU.

In [ ]:
from torchvision.ops import box_iou
from torch import torch

def center_to_corner_box(bbox):
    x_max = bbox[:, 0] + bbox[:, 2]
    y_max = bbox[:, 1] + bbox[:, 3]
    result = bbox.copy()
    result[:, 2] = x_max
    result[:, 3] = y_max
    return result

def generate_anchores(image_size, scales, aspect_ratios, feature_map_size):
    anchor_boxes_centers = []  # For center coordinates
    anchor_boxes_corners = []  # For corner coordinates
    
    step_size = image_size / feature_map_size  # Size of one grid cell
    
    for x in range(feature_map_size):
        for y in range(feature_map_size):
            center_x = (x + 0.5) * step_size
            center_y = (y + 0.5) * step_size
            
            for scale in scales:
                for aspect_ratio in aspect_ratios:
                    box_height = image_size * scale / np.sqrt(aspect_ratio)
                    box_width = image_size * scale * np.sqrt(aspect_ratio)
                    
                    # Center coordinates with width and height
                    anchor_boxes_centers.append([center_x, center_y, box_width, box_height])
                    
                    # Convert to corner coordinates
                    x_min = center_x - box_width / 2
                    y_min = center_y - box_height / 2
                    x_max = center_x + box_width / 2
                    y_max = center_y + box_height / 2
                    anchor_boxes_corners.append([x_min, y_min, x_max, y_max])
    
    return np.array(anchor_boxes_centers), np.array(anchor_boxes_corners)

def analyze_anchor_boxes(bboxes_corners, anchore_corners, iou_cutoffs=[0.25, 0.5, 0.75, 0.9]):

    num_boxes = bboxes_corners.shape[0]

    # convert to 
    bboxes_corners = torch.tensor(bboxes_corners)
    anchore_corners = torch.tensor(anchore_corners)

    ious = box_iou(bboxes_corners, anchore_corners)

    for iou_cutoff in iou_cutoffs:
        print(f"\nAnalyzing cutoff IoU = {iou_cutoff}")

        positive_mask = ious >= iou_cutoff

        pos_per_sample = torch.sum(positive_mask, dim=1)
        avg_pos_per_sample = torch.mean(pos_per_sample.float()).item()
        coverage = torch.sum(pos_per_sample > 0).item() / num_boxes

        print(f"\t-Average Anchors Per Box: {avg_pos_per_sample}")
        print(f"\t-Ratio of Covered Boxes: {coverage}")

In [ ]:
with open('./../datasets/tumor-segmentation/test/_annotations.coco.json', "r") as f:
    train_annotations = json.load(f)
train_bboxes = np.round(np.array([img['bbox'] for img in train_annotations['annotations']])).astype(int)
train_bboxes_corners = center_to_corner_box(train_bboxes)

In [ ]:
image_size = 640
scales = [0.1, 0.2, 0.3]
aspect_ratios = [1]
feature_map_size = 20
anchore_centers, anchor_corners = generate_anchores(image_size, scales, aspect_ratios, feature_map_size)

print(f"Anchored boxes: {len(anchore_centers)}")
analyze_anchor_boxes(train_bboxes_corners, anchor_corners)

In [ ]:
image_size = 640
scales = [0.1, 0.2, 0.3]
aspect_ratios = [0.66667, 1, 1.5]
feature_map_size = 20
anchore_centers, anchor_corners = generate_anchores(image_size, scales, aspect_ratios, feature_map_size)

print(f"Anchored boxes: {len(anchore_centers)}")
analyze_anchor_boxes(train_bboxes_corners, anchor_corners)

In [ ]:
image_size = 640
scales = [0.1, 0.175, 0.25, 0.3]
aspect_ratios = [1]
feature_map_size = 20
anchore_centers, anchor_corners = generate_anchores(image_size, scales, aspect_ratios, feature_map_size)

print(f"Anchored boxes: {len(anchore_centers)}")
analyze_anchor_boxes(train_bboxes_corners, anchor_corners)

In [ ]:
analyze_anchor_boxes(train_bboxes_corners, anchor_corners)